<a href="https://colab.research.google.com/github/woojung02/SSAC_AI/blob/main/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch  # PyTorch 라이브러리 임포트 (딥러닝 프레임워크)
import torch.nn as nn  # 신경망 모듈
import torch.optim as optim  # 최적화 알고리즘
import torchvision  # 이미지 데이터셋과 전처리 도구
import torchvision.transforms as transforms  # 이미지 변환 함수들

# GPU 사용 가능하면 'cuda', 아니면 'cpu'를 사용하기 위한 장치(device) 지정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# CIFAR-10 학습 데이터셋에 적용할 이미지 전처리 및 데이터 증강 정의
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  # 원본 이미지 주변에 4픽셀 패딩 후 32x32 영역 랜덤 크롭 (과적합 방지)
    transforms.RandomHorizontalFlip(),  # 좌우 반전 (데이터 다양성 증대)
    transforms.ToTensor(),  # 이미지를 텐서로 변환 (0~1 값)
    transforms.Normalize((0.4914, 0.4822, 0.4465),  # RGB 3채널 평균값으로 정규화(직접 계산해서 넣어준다. 이유:입력데이터가 일정한 값을 가지면 안정도,속도가 올라간다.)
                         (0.2023, 0.1994, 0.2010))  # RGB 3채널 표준편차로 정규화(평균값으로 정규화한 이유와 같은 이유)
])

# 테스트 데이터셋 전처리
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

# CIFAR-10 학습 데이터셋 다운로드 및 전처리 적용(CIFAR-10은 50000(학습용)+10000(테스트용)으로 구성 되어있고,32*32의 RGB로 구성된다.10개의 클라스로 분류되고 (배,차,고양이,강아지)등 크기가 작은 이미지로 이루어져 있다.)
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
# 학습 데이터셋을 배치 크기 128, 셔플 옵션 켜서 로드(shuffle이란 배치 데이터를 무작위로 섞는거,사용 이유:데이터가 순서대로 학습하면 편향될수 있고 ,과적합을 줄일수 있다. 즉 순서에 의존하는것을 막을수 있다.)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

# CIFAR-10 테스트 데이터셋 다운로드 및 전처리 적용
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)
# 테스트 데이터셋 배치 크기 100, 셔플 없음(셔플=false인 이유:학습 단계가 아닌 테스트 단계이므로 일관된 평가를 해야하니 사용 하지 않음)
testloader = torch.utils.data.DataLoader(testset, batch_size=100,
                                         shuffle=False, num_workers=2)

#  ResidualBlock 정의(잔차 학습은 입력함수x와 변환함수 H(x)의 차이를 학습한다.이유:기울기가 0에 가까워지는 문제(기울기 소실)을 해결할수 있다)
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        # 첫번째 3x3 합성곱, stride와 padding=1로 공간 크기 조절, bias는 batchnorm 때문에 False
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)  # 배치 정규화
        self.relu = nn.ReLU(inplace=True)  # ReLU 활성화 함수, 메모리 절약 inplace=True
        # 두번째 3x3 합성곱, stride=1로 공간 크기 유지
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample  # 입력과 출력 차원 다르면 맞춰주는 레이어

    def forward(self, x):
        identity = x  # 입력을 저장 (skip connection 용)

        out = self.conv1(x)  # 첫 합성곱 수행
        out = self.bn1(out)  # 배치 정규화
        out = self.relu(out)  # ReLU 활성화

        out = self.conv2(out)  # 두번째 합성곱 수행
        out = self.bn2(out)  # 배치 정규화

        # 입력과 출력 크기가 다르면 downsample로 맞춤
        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity  # skip connection: 입력을 결과에 더함
        out = self.relu(out)  # ReLU 활성화

        return out

# 전체 ResNet-18 모델 정의
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 64  # 초기 입력 채널 수 (conv1 출력 채널)

        # CIFAR-10은 32x32 이미지라서 kernel=3, stride=1, padding=1 사용 (이미지 크기 유지)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        # CIFAR-10 특성상 maxpool 생략 가능 (원하면 추가 가능)
        # self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # 각 레이어 별로 ResidualBlock 반복 횟수와 출력 채널 수 지정
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)  # stride=2로 다운샘플링 (절반 크기)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # 공간 크기를 1x1로 줄임
        self.fc = nn.Linear(512, num_classes)  # 최종 클래스 분류용 완전연결층

    # ResidualBlock 레이어를 여러 개 쌓는 함수
    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        # 입력 채널 수와 출력 채널 수가 다르거나 stride가 1이 아니면 downsample 적용
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )
        layers = []
        # 첫 번째 블록에서는 stride, downsample 적용
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        # 두 번째 이후 블록들은 stride=1, downsample 없음
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))

        return nn.Sequential(*layers)  # nn.Sequential로 묶음

    def forward(self, x):
        x = self.conv1(x)  # 초기 conv 연산
        x = self.bn1(x)
        x = self.relu(x)
        # CIFAR-10은 maxpool 생략(이유:데이터 크기가 작아서 크기를 줄이면 정보가 많이 소실됨)
        # x = self.maxpool(x)

        x = self.layer1(x)  # ResidualBlock 층 쌓기
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)  # 공간 크기 1x1로 줄이기
        x = torch.flatten(x, 1)  # 배치 차원 제외하고 flatten
        x = self.fc(x)  # 최종 완전연결층 통과

        return x

# ResNet-18 생성 함수: ResidualBlock 2개씩 4개 층으로 구성
def resnet18(num_classes=10):
    return ResNet(ResidualBlock, [2, 2, 2, 2], num_classes)

# 모델 생성 및 device 할당
model = resnet18(num_classes=10).to(device)

# 손실 함수로 크로스엔트로피 설정
criterion = nn.CrossEntropyLoss()

# SGD 옵티마이저 설정: 학습률 0.1, 모멘텀 0.9, 가중치 감쇠 5e-4
optimizer = optim.SGD(model.parameters(), lr=0.1,
                      momentum=0.9, weight_decay=5e-4)

# 학습률 스케줄러: 30, 60, 90 에폭마다 0.1배씩 감소(학습률learning rate를 조정 하는것,이유:학습률이 크면 발산하고 작으면 너무 느리니 적당한 값을 찾는것은 중요 0.1배씩 감소 한다는것은 학습률이 1이면 30번째 에포치 부터는 학습률 0.1 60은 0.01 90은 0.001이다.
scheduler = optim.lr_scheduler.MultiStepLR(optimizer,
                                           milestones=[30, 60, 90], gamma=0.1)

# 학습 함수
def train(epoch):
    model.train()  # 학습 모드 설정 (드롭아웃, 배치정규화 등 활성화)
    running_loss = 0.0
    correct = 0
    total = 0
    print(f"Epoch {epoch} 시작")
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)  # GPU로 데이터 이동

        optimizer.zero_grad()  # 기울기 초기화
        outputs = model(inputs)  # 순전파
        loss = criterion(outputs, targets)  # 손실 계산
        loss.backward()  # 역전파로 기울기 계산
        optimizer.step()  # 가중치 갱신

        running_loss += loss.item()
        _, predicted = outputs.max(1)  # 예측 결과
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # 100 미니배치마다 손실과 정확도 출력
        if batch_idx % 100 == 99:
            print(f"  Batch {batch_idx+1}, Loss: {running_loss / 100:.3f}, Accuracy: {100.*correct/total:.2f}%")
            running_loss = 0.0

# 테스트 함수
def test(epoch):
    model.eval()  # 평가 모드 설정 (드롭아웃 등 비활성화)
    correct = 0
    total = 0
    test_loss = 0.0
    with torch.no_grad():  # 기울기 계산 안함
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    print(f"Epoch {epoch} 테스트: Loss: {test_loss/len(testloader):.3f}, Accuracy: {100.*correct/total:.2f}%")

# 총 학습 횟수 (에폭) 설정
num_epochs = 100

# 학습 및 테스트 반복
for epoch in range(1, num_epochs + 1):
    train(epoch)
    test(epoch)
    scheduler.step()  # 학습률 스케줄러 업데이트
